In [5]:
# Fronteira Eficiente - Análise de Portfólio
# Autor: Robaina
# Conversão para Python/Jupyter

# Configurações e imports
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar
import warnings
warnings.filterwarnings('ignore')

# Configurações de exibição
pd.set_option('display.float_format', '{:.4f}'.format)
np.random.seed(42)

# Parâmetros
tickers = [
    "HASH11.SA",   # Bitcoin
    "IVVB11.SA",
    "GOLD11.SA",   # Ouro / Commodities
    "BOVA11.SA",   # Índice Bovespa
    "IMAB11.SA",   # Renda Fixa / Tesouro IPCA
    "FIXA11.SA",   # Renda Fixa / CDI
]

start_date = "2020-01-01"
end_date = "2025-06-01"
risk_free = 0.15  # Taxa livre de risco anual (15% a.a.)

print("Configurações carregadas com sucesso!")
print(f"Ativos: {tickers}")
print(f"Período: {start_date} a {end_date}")
print(f"Taxa livre de risco: {risk_free*100:.1f}% a.a.")

# =============================================================================
# 1) BAIXAR DADOS (Yahoo Finance)
# =============================================================================

print("\n" + "="*60)
print("1) BAIXANDO DADOS DO YAHOO FINANCE")
print("="*60)

def baixar_dados(tickers, start_date, end_date):
    """Baixa dados de preços ajustados do Yahoo Finance"""
    prices_data = {}
    mensagens = []
    
    for ticker in tickers:
        try:
            print(f"📥 Tentando baixar {ticker}...")
            
            # Tenta baixar com diferentes configurações
            data = yf.download(ticker, 
                             start=start_date, 
                             end=end_date, 
                             progress=False,
                             interval='1d',
                             auto_adjust=True,
                             prepost=True,
                             threads=True)
            
            if not data.empty and 'Adj Close' in data.columns:
                prices_data[ticker] = data['Adj Close']
                print(f"✓ {ticker}: {len(data)} observações baixadas")
            elif not data.empty and 'Close' in data.columns:
                # Fallback para Close se Adj Close não estiver disponível
                prices_data[ticker] = data['Close']
                print(f"✓ {ticker}: {len(data)} observações (usando Close)")
            else:
                mensagens.append(f"✗ {ticker}: DataFrame vazio ou sem coluna de preços")
                
        except Exception as e:
            mensagens.append(f"✗ {ticker}: Erro - {str(e)}")
            print(f"✗ {ticker}: Falhou - {str(e)}")
    
    if mensagens:
        print(f"\n⚠️  Avisos ({len(mensagens)} problemas):")
        for msg in mensagens:
            print(f"  {msg}")
    
    if not prices_data:
        print("\n❌ ERRO: Nenhum dado foi baixado!")
        print("Possíveis soluções:")
        print("1. Verifique sua conexão com a internet")
        print("2. Tente tickers diferentes (ex: AAPL, MSFT, GOOGL)")
        print("3. Ajuste as datas (período muito recente ou antigo)")
        
        # Dados de exemplo como fallback
        print("\n🔄 Gerando dados sintéticos para demonstração...")
        np.random.seed(42)
        dates = pd.date_range(start=start_date, end=end_date, freq='D')
        dates = dates[dates.weekday < 5]  # Apenas dias úteis
        
        for ticker in tickers:
            # Simula uma série de preços com random walk
            n_days = len(dates)
            returns = np.random.normal(0.0005, 0.02, n_days)  # ~0.125% retorno diário médio
            prices_sim = 100 * np.exp(np.cumsum(returns))  # Começa em R$ 100
            prices_data[ticker] = pd.Series(prices_sim, index=dates)
        
        print(f"✓ Dados sintéticos gerados para {len(tickers)} ativos")
        print("⚠️  ATENÇÃO: Usando dados simulados apenas para demonstração!")
    
    # Consolida em DataFrame e remove NAs
    prices = pd.DataFrame(prices_data).dropna()
    
    if prices.empty:
        raise ValueError("Erro crítico: Não foi possível criar DataFrame de preços!")
    
    return prices

# Baixar dados
try:
    prices = baixar_dados(tickers, start_date, end_date)
except Exception as e:
    print(f"❌ Erro crítico: {e}")
    # Se tudo falhar, cria dados mínimos para o exemplo funcionar
    print("🔄 Criando exemplo mínimo com dados fictícios...")
    
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    dates = dates[dates.weekday < 5][:500]  # 500 dias úteis
    
    np.random.seed(42)
    prices_dict = {}
    for i, ticker in enumerate(tickers):
        base_price = 50 + i * 20  # Preços base diferentes
        returns = np.random.normal(0.0003, 0.015 + i*0.003, len(dates))
        prices_dict[ticker] = base_price * np.exp(np.cumsum(returns))
    
    prices = pd.DataFrame(prices_dict, index=dates)
    print("✓ Dados de exemplo criados com sucesso!")

# Mostrar últimos preços
print(f"\n📊 ÚLTIMA COTAÇÃO DISPONÍVEL ({prices.index[-1].strftime('%Y-%m-%d')}):")
print("="*50)
ultima_cotacao = pd.DataFrame({
    'Ativo': prices.columns,
    'Preço (R$)': prices.iloc[-1].values
}).round(2)
print(ultima_cotacao.to_string(index=False))

# =============================================================================
# 2) RETORNOS E ESTATÍSTICAS BÁSICAS
# =============================================================================

print("\n" + "="*60)
print("2) RETORNOS E ESTATÍSTICAS BÁSICAS")
print("="*60)

def calcular_metricas_anuais(returns, risk_free_rate=0.03):
    """Calcula métricas anualizadas dos ativos"""
    # Retorno anual (média geométrica)
    retorno_anual = (1 + returns.mean())**252 - 1
    
    # Volatilidade anual
    volatilidade_anual = returns.std() * np.sqrt(252)
    
    # Sharpe ratio
    sharpe = (retorno_anual - risk_free_rate) / volatilidade_anual
    
    return retorno_anual, volatilidade_anual, sharpe

# Calcular retornos logarítmicos
returns = np.log(prices / prices.shift(1)).dropna()

# Covariância anualizada
cov_anual = returns.cov() * 252

# Métricas anuais
ret_anual, vol_anual, sharpe_anual = calcular_metricas_anuais(returns, risk_free)

# Tabela de estatísticas
stats_df = pd.DataFrame({
    'Ativo': returns.columns,
    'Retorno Anual (%)': (ret_anual * 100).round(2),
    'Risco Anual (%)': (vol_anual * 100).round(2),
    'Sharpe (a.a.)': sharpe_anual.round(3)
})

print("📈 ESTATÍSTICAS POR ATIVO (anualizadas):")
print("="*45)
print(stats_df.to_string(index=False))

# Matriz de correlação
print(f"\n📊 MATRIZ DE CORRELAÇÃO:")
print("="*30)
corr_matrix = returns.corr()
print(corr_matrix.round(3))

# Gráfico de correlação
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='RdYlBu_r', center=0, 
            square=True, fmt='.3f', cbar_kws={'shrink': 0.8})
plt.title('Matriz de Correlação entre Ativos', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

# =============================================================================
# 3) PORTFÓLIOS DE REFERÊNCIA
# =============================================================================

print("\n" + "="*60)
print("3) PORTFÓLIOS DE REFERÊNCIA")
print("="*60)

class PortfolioOptimizer:
    def __init__(self, returns, cov_matrix, risk_free_rate=0.03):
        self.returns = returns
        self.cov_matrix = cov_matrix
        self.expected_returns = (1 + returns.mean())**252 - 1
        self.risk_free_rate = risk_free_rate
        self.n_assets = len(self.expected_returns)
    
    def portfolio_metrics(self, weights):
        """Calcula métricas do portfólio"""
        weights = np.array(weights)
        weights = weights / weights.sum()  # Normaliza
        
        ret = np.sum(weights * self.expected_returns)
        vol = np.sqrt(np.dot(weights.T, np.dot(self.cov_matrix, weights)))
        sharpe = (ret - self.risk_free_rate) / vol
        
        return ret, vol, sharpe
    
    def minimize_volatility(self):
        """Otimiza para mínima volatilidade"""
        def objective(weights):
            _, vol, _ = self.portfolio_metrics(weights)
            return vol
        
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bounds = tuple((0, 1) for _ in range(self.n_assets))
        
        result = minimize(objective, 
                         x0=np.ones(self.n_assets) / self.n_assets,
                         method='SLSQP',
                         bounds=bounds,
                         constraints=constraints)
        
        return result.x / result.x.sum()
    
    def maximize_sharpe(self):
        """Otimiza para máximo Sharpe ratio"""
        def objective(weights):
            _, _, sharpe = self.portfolio_metrics(weights)
            return -sharpe  # Minimiza o negativo = maximiza
        
        constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
        bounds = tuple((0, 1) for _ in range(self.n_assets))
        
        result = minimize(objective,
                         x0=np.ones(self.n_assets) / self.n_assets,
                         method='SLSQP',
                         bounds=bounds,
                         constraints=constraints)
        
        return result.x / result.x.sum()

# Inicializar otimizador
optimizer = PortfolioOptimizer(returns, cov_anual, risk_free)

# Otimizar portfólios
weights_minvol = optimizer.minimize_volatility()
weights_sharpe = optimizer.maximize_sharpe()

# Métricas dos portfólios otimizados
ret_minvol, vol_minvol, sharpe_minvol = optimizer.portfolio_metrics(weights_minvol)
ret_sharpe, vol_sharpe, sharpe_sharpe = optimizer.portfolio_metrics(weights_sharpe)

# Composição dos portfólios
def criar_tabela_composicao(weights, portfolio_name, ret, vol, sharpe):
    df = pd.DataFrame({
        'Ativo': returns.columns,
        'Peso (%)': (weights * 100).round(2)
    }).sort_values('Peso (%)', ascending=False)
    
    print(f"\n💼 {portfolio_name}")
    print(f"   Retorno: {ret*100:.2f}% | Risco: {vol*100:.2f}% | Sharpe: {sharpe:.3f}")
    print("-" * 55)
    print(df.to_string(index=False))
    
    return df

comp_minvol = criar_tabela_composicao(weights_minvol, "MÍNIMA VOLATILIDADE", 
                                     ret_minvol, vol_minvol, sharpe_minvol)

comp_sharpe = criar_tabela_composicao(weights_sharpe, "MÁXIMO SHARPE", 
                                     ret_sharpe, vol_sharpe, sharpe_sharpe)

# =============================================================================
# 4) FRONTEIRA EFICIENTE
# =============================================================================

print("\n" + "="*60)
print("4) FRONTEIRA EFICIENTE")
print("="*60)

def gerar_fronteira_eficiente(optimizer, n_pontos=50):
    """Gera pontos da fronteira eficiente"""
    # Limites de retorno
    ret_min = ret_minvol
    ret_max = optimizer.expected_returns.max()
    
    target_returns = np.linspace(ret_min, ret_max, n_pontos)
    
    frontier_results = []
    
    for target_ret in target_returns:
        try:
            # Otimização com restrição de retorno alvo
            def objective(weights):
                _, vol, _ = optimizer.portfolio_metrics(weights)
                return vol
            
            constraints = [
                {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Soma = 1
                {'type': 'eq', 'fun': lambda x: np.sum(x * optimizer.expected_returns) - target_ret}  # Retorno alvo
            ]
            
            bounds = tuple((0, 1) for _ in range(optimizer.n_assets))
            
            result = minimize(objective,
                            x0=np.ones(optimizer.n_assets) / optimizer.n_assets,
                            method='SLSQP',
                            bounds=bounds,
                            constraints=constraints)
            
            if result.success:
                weights = result.x / result.x.sum()
                ret, vol, sharpe = optimizer.portfolio_metrics(weights)
                frontier_results.append([ret, vol, sharpe] + list(weights))
            
        except:
            continue
    
    # Criar DataFrame da fronteira
    columns = ['Return', 'Risk', 'Sharpe'] + [f'w_{asset}' for asset in returns.columns]
    frontier_df = pd.DataFrame(frontier_results, columns=columns)
    
    return frontier_df

print("🔄 Calculando fronteira eficiente...")
frontier = gerar_fronteira_eficiente(optimizer, n_pontos=40)

# Gráfico da fronteira eficiente
plt.figure(figsize=(12, 8))

# Linha da fronteira
plt.plot(frontier['Risk']*100, frontier['Return']*100, 
         'b-', linewidth=2, label='Fronteira Eficiente')

# Pontos de referência
plt.scatter(vol_minvol*100, ret_minvol*100, 
           c='green', s=100, marker='D', label='Mínima Volatilidade', zorder=5)
plt.scatter(vol_sharpe*100, ret_sharpe*100, 
           c='red', s=100, marker='^', label='Máximo Sharpe', zorder=5)

# Ativos individuais
plt.scatter(vol_anual*100, ret_anual*100, 
           c='orange', s=60, alpha=0.7, label='Ativos Individuais')

# Anotações dos ativos
for i, asset in enumerate(returns.columns):
    plt.annotate(asset, (vol_anual.iloc[i]*100, ret_anual.iloc[i]*100),
                xytext=(5, 5), textcoords='offset points', fontsize=9)

plt.xlabel('Risco (% a.a.)', fontsize=12)
plt.ylabel('Retorno (% a.a.)', fontsize=12)
plt.title('Fronteira Eficiente - Otimização de Portfólio\n'
          'Análise sem vendas a descoberto (w ≥ 0)', fontsize=14, pad=20)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# =============================================================================
# 5) RESUMO FINAL
# =============================================================================

print("\n" + "="*60)
print("5) RESUMO FINAL")
print("="*60)

resumo_df = pd.DataFrame({
    'Portfolio': ['Mínima Volatilidade', 'Máximo Sharpe'],
    'Retorno (%)': [f"{ret_minvol*100:.2f}", f"{ret_sharpe*100:.2f}"],
    'Risco (%)': [f"{vol_minvol*100:.2f}", f"{vol_sharpe*100:.2f}"],
    'Sharpe': [f"{sharpe_minvol:.3f}", f"{sharpe_sharpe:.3f}"]
})

print("📊 RESUMO DOS PORTFÓLIOS DE REFERÊNCIA (anualizados):")
print("="*55)
print(resumo_df.to_string(index=False))

print(f"\n🔍 INSIGHTS:")
print(f"• A fronteira eficiente mostra as melhores combinações risco-retorno")
print(f"• Portfolio de Mínima Volatilidade: menor risco possível")
print(f"• Portfolio de Máximo Sharpe: melhor retorno ajustado ao risco")
print(f"• Taxa livre de risco utilizada: {risk_free*100:.1f}% a.a.")
print(f"• Período de análise: {start_date} a {end_date}")
print(f"• Total de observações: {len(returns)} dias úteis")

print("\n" + "="*60)
print("ANÁLISE CONCLUÍDA! ✅")
print("="*60)

Configurações carregadas com sucesso!
Ativos: ['HASH11.SA', 'IVVB11.SA', 'GOLD11.SA', 'BOVA11.SA', 'IMAB11.SA', 'FIXA11.SA']
Período: 2020-01-01 a 2025-06-01
Taxa livre de risco: 15.0% a.a.

1) BAIXANDO DADOS DO YAHOO FINANCE
📥 Tentando baixar HASH11.SA...
✓ HASH11.SA: 1023 observações (usando Close)
📥 Tentando baixar IVVB11.SA...
✓ IVVB11.SA: 1346 observações (usando Close)
📥 Tentando baixar GOLD11.SA...
✓ GOLD11.SA: 1107 observações (usando Close)
📥 Tentando baixar BOVA11.SA...
✓ BOVA11.SA: 1346 observações (usando Close)
📥 Tentando baixar IMAB11.SA...
✓ IMAB11.SA: 1346 observações (usando Close)
📥 Tentando baixar FIXA11.SA...
✓ FIXA11.SA: 1087 observações (usando Close)
❌ Erro crítico: If using all scalar values, you must pass an index
🔄 Criando exemplo mínimo com dados fictícios...
✓ Dados de exemplo criados com sucesso!

📊 ÚLTIMA COTAÇÃO DISPONÍVEL (2021-11-30):
    Ativo  Preço (R$)
HASH11.SA     61.1500
IVVB11.SA    108.3000
GOLD11.SA    326.6500
BOVA11.SA    190.3300
IMAB11.SA 

In [6]:
ativos = [
    "HASH11.SA",   # Bitcoin
    "IVVB11.SA",
    "GOLD11.SA",   # Ouro / Commodities
    "BOVA11.SA",   # Índice Bovespa
    "IMAB11.SA",   # Renda Fixa / Tesouro IPCA
    "FIXA11.SA",   # Renda Fixa / CDI
]
respostas = {'objetivo': 'arrojado', 'max_ativos': '4', 'horizonte': 'curto', 'Bitcoin': 'X%'}
if respostas['Bitcoin'] == '0%':
    ativos.pop(0)

In [7]:
"""
Módulo de decisão: define a regra para selecionar ativos com base nas respostas do usuário.
"""
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import yfinance as yf
import matplotlib
matplotlib.use('Agg')  # Define o backend não interativo
import matplotlib.pyplot as plt
import os

def baixar_dados(tickers, start_date='2022-01-01', end_date='2024-01-01'):
    """
    Baixa preços ajustados dos ativos da B3 usando yfinance.
    """
    # Garantir que todos os tickers sejam strings
    tickers = [str(ticker) for ticker in tickers]

    # Baixar os preços ajustados
    prices = yf.download(tickers, start=start_date, end=end_date, progress=False)['Close']
    prices = prices.dropna()

    return prices

def calcular_metricas(returns):
    """
    Calcula métricas para cada ativo: retorno médio e volatilidade.
    """
    retorno_medio = returns.mean() * 252  # Retorno anualizado
    volatilidade = returns.std() * np.sqrt(252)  # Volatilidade anualizada
    metricas = pd.DataFrame({
        'Retorno Médio': retorno_medio,
        'Volatilidade': volatilidade
    })
    return metricas

def clusterizar_ativos(metricas, n_clusters):
    """
    Aplica K-Means para agrupar os ativos com base em suas métricas.
    Gera gráficos da clusterização e salva na pasta static/.
    """
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    metricas['Cluster'] = kmeans.fit_predict(metricas)

    # Gerar gráfico da clusterização
    plt.figure(figsize=(10, 6))
    for cluster in range(n_clusters):
        cluster_data = metricas[metricas['Cluster'] == cluster]
        plt.scatter(cluster_data['Volatilidade'], cluster_data['Retorno Médio'], label=f'Cluster {cluster}')
    
    # Destacar os ativos escolhidos
    ativos_escolhidos = metricas.groupby('Cluster').apply(lambda x: x.sample(1, random_state=42))
    plt.scatter(ativos_escolhidos['Volatilidade'], ativos_escolhidos['Retorno Médio'], 
                color='black', label='Ativos Escolhidos', edgecolor='white', s=100)

    plt.xlabel('Volatilidade (Risco)')
    plt.ylabel('Retorno Médio')
    plt.title('Clusterização dos Ativos')
    plt.legend()
    plt.grid(True)

    # Salvar o gráfico na pasta static/
    os.makedirs('static', exist_ok=True)
    plt.savefig('static/clusterizacao.png')
    plt.close()

    return metricas

def escolher_ativos(respostas):
    """
    Recebe as respostas do formulário e retorna uma lista de tickers da B3.
    Utiliza clusterização para selecionar os ativos.
    """
    # Lista de ativos disponíveis


tickers = [
    "PETR4.SA",  # Petrobras PN
    "VALE3.SA",  # Vale ON
    "ITUB4.SA",  # Itaú Unibanco PN
    "BBDC4.SA",  # Bradesco PN
    "ABEV3.SA"   # Ambev ON
]

    # Número de ativos desejados (entre 1 e 25)
n_ativos = 5  # Default: 5 ativos

    # Baixar dados de preços
prices = baixar_dados(tickers)
prices

Ticker,ABEV3.SA,BBDC4.SA,ITUB4.SA,PETR4.SA,VALE3.SA
Date,,,,,
2022-01-03,12.7465,13.6470,15.5965,10.6026,55.8081
2022-01-04,12.7299,13.7364,16.0388,10.6427,55.1498
2022-01-05,12.4804,13.6393,15.7343,10.2309,55.6721
2022-01-06,12.2809,13.8336,16.0533,10.2236,56.7955
2022-01-07,12.0813,14.0347,16.4086,10.2709,60.1010
...,...,...,...,...,...
2023-12-21,12.6182,14.3977,25.8900,27.5077,65.5042
2023-12-22,12.6274,14.4423,26.3006,27.7723,65.0106
2023-12-26,12.6827,14.4852,26.5690,28.2183,65.2404


In [28]:
import pandas as pd
import yfinance as yf
import numpy as np
def baixar_dados(tickers, start_date='2020-01-01', end_date='2025-01-01'):

    data = yf.download(tickers, start=start_date, end=end_date, progress=False)

    prices = data['Close']

    prices = prices.dropna(axis=1, how='any')

    returns = prices.pct_change().dropna()

    #Pra que eu vou usar isso?
    retorno_acumulado = (1 + returns).cumprod()
    final_retorno_acumulado = retorno_acumulado.iloc[-1] if not retorno_acumulado.empty else pd.Series(1, index=data.columns)


    return data,returns,final_retorno_acumulado

tickers = ['BOVA11.SA', 'IVVB11.SA', 'SMAL11.SA']

df = baixar_dados(tickers)[0]
df

C:\Users\gabri\AppData\Local\Temp\ipykernel_2880\4076068935.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date, progress=False)


Price            Close                                High              \
Ticker       BOVA11.SA   IVVB11.SA   SMAL11.SA   BOVA11.SA   IVVB11.SA   
Date                                                                     
2020-01-02  114.239998  140.600006  139.500000  114.239998  140.600006   
2020-01-03  113.800003  140.699997  140.899994  114.500000  141.169998   
2020-01-06  112.589996  141.199997  139.100006  113.449997  141.460007   
2020-01-07  112.239998  141.300003  139.399994  112.900002  142.080002   
2020-01-08  111.949997  141.550003  138.199997  113.099998  142.000000   
...                ...         ...         ...         ...         ...   
2024-12-20  119.050003  403.450012   88.980003  119.089996  407.320007   
2024-12-23  117.699997  413.500000   85.699997  118.599998  414.290009   
2024-12-26  117.959999  417.200012   85.470001  118.570000  417.489990   
2024-12-27  117.260002  412.899994   85.650002  118.470001  417.880005   
2024-12-30  117.269997  407.700012   85.949997  118.000000  412.429993   

Price                          Low                                Open  \
Ticker       SMAL11.SA   BOVA11.SA   IVVB11.SA   SMAL11.SA   BOVA11.SA   
Date                                                                     
2020-01-02  139.500000  112.129997  139.100006  138.000000  112.449997   
2020-01-03  140.899994  112.800003  140.000000  137.130005  112.930000   
2020-01-06  139.660004  112.019997  140.029999  137.000000  113.000000   
2020-01-07  139.710007  111.589996  140.750000  138.179993  112.900002   
2020-01-08  139.789993  111.400002  141.009995  137.699997  112.650002   
...                ...         ...         ...         ...         ...   
2024-12-20   89.059998  117.660004  394.510010   86.139999  118.000000   
2024-12-23   88.000000  117.529999  406.609985   85.699997  118.599998   
2024-12-26   86.180000  117.389999  413.500000   85.150002  117.900002   
2024-12-27   86.500000  117.019997  410.750000   85.440002  118.300003   
2024-12-30   87.080002  117.080002  405.269989   85.430000  117.430000   

Price                                 Volume                      
Ticker       IVVB11.SA   SMAL11.SA BOVA11.SA IVVB11.SA SMAL11.SA  
Date                                                              
2020-01-02  139.509995  138.000000   5684380    152740    631240  
2020-01-03  140.399994  138.000000   6602450    257310    401120  
2020-01-06  140.539993  139.000000   6771940    121770    418470  
2020-01-07  141.610001  139.500000   6041900    109590    160520  
2020-01-08  141.300003  139.410004   6472610    244850    555410  
...                ...         ...       ...       ...       ...  
2024-12-20  396.589996   87.190002   6972532    329015   3952340  
2024-12-23  407.730011   88.000000   6218302    337963   1510757  
2024-12-26  414.519989   85.709999   3231104    171647   1831351  
2024-12-27  416.269989   85.949997   6917466    275964   1141093  
2024-12-30  410.690002   86.000000   6590812    276116   1409576  

[1244 rows x 15 columns]

In [ ]:
from sklearn.impute import SimpleImputer
def features_para_cluster(data,retorno,final_retorno_acumulado):

    features = pd.DataFrame(index=data['Close'].columns)

    features['Retorno_Medio'] = retorno.mean() * 252

    features['Volatilidade'] = retorno.std() * np.sqrt(252)

    features['Sharpe'] = np.where(
        features['Volatilidade'] > 0,
        features['Retorno_Medio'] / features['Volatilidade'],
        0
        )

    features['Retorno_Acumulado'] = (final_retorno_acumulado - 1).fillna(0)

    features['Skewness'] = retorno.skew().fillna(0)

    features['Kurtosis'] = retorno.kurtosis().fillna(0)


    cummax = data.cummax()
    drawdown = (data - cummax) / cummax
    features['Max_Drawdown'] = drawdown.min().fillna(0)


    # Correlação média
    corr_matrix = retorno.corr()
    n = len(corr_matrix)
    if n > 1:
        corr_sum = corr_matrix.values.sum() - n  # Subtrair diagonal
        features['Corr_Media'] = corr_sum / (n * (n - 1))
    else:
        features['Corr_Media'] = 0

    imputer = SimpleImputer(strategy='mean')
    features_scaled = imputer.fit_transform(features)


    features = features.replace([np.inf, -np.inf], np.nan)
    features = features.fillna(0)

    return features, features_scaled

data, returns, final = baixar_dados(tickers)

features, features_scaled = features_para_cluster(data, returns, final)
features




C:\Users\gabri\AppData\Local\Temp\ipykernel_2880\4076068935.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date, progress=False)
C:\Users\gabri\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\impute\_base.py:653: UserWarning: Skipping features without any observed values: ['Max_Drawdown']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


,Retorno_Medio,Volatilidade,Sharpe,Retorno_Acumulado,Skewness,Kurtosis,Max_Drawdown,Corr_Media
Ticker,,,,,,,,
BOVA11.SA,0.039602,0.260286,0.152149,0.026523,-0.967831,17.580253,0.0,0.416212
IVVB11.SA,0.240578,0.222593,1.080801,1.899715,0.442009,7.259936,0.0,0.416212
SMAL11.SA,-0.046465,0.318633,-0.145828,-0.383871,-1.191528,11.766888,0.0,0.416212


In [143]:
def cluster_cotovelo(features_scaled):

    n_samples = len(features_scaled)
    min_k = 2
    max_k = min(8, max(2, n_samples - 1))

    if max_k < min_k:
        optimal_k = 2
        print(f"⚠️ Poucos dados. Usando k={optimal_k}")
        inertias = []
        silhouette_scores = []
        K = [optimal_k]
    else:
        K = list(range(min_k, max_k + 1))
        inertias = []
        silhouette_scores = []
        
        for k in K:
            try:
                kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
                kmeans.fit(features_scaled)
                inertias.append(kmeans.inertia_)
                
                if k < n_samples:
                    score = silhouette_score(features_scaled, kmeans.labels_)
                    silhouette_scores.append(score)
            except:
                inertias.append(float('inf'))
                silhouette_scores.append(0)
        
        # Encontrar cotovelo
        if len(inertias) >= 3:
            # Método simples: escolher k onde a redução de inércia diminui
            reductions = np.diff(inertias)
            if len(reductions) > 0:
                optimal_k = min_k + np.argmin(reductions) + 1
            else:
                optimal_k = 3
        else:
            optimal_k = min(3, max_k)
        
        optimal_k = max(min_k, min(optimal_k, max_k))
        return optimal_k, inertias, silhouette_scores
print("\n📈 Aplicando Método do Cotovelo...")
print(f"✅ Número ótimo de clusters: {cluster_cotovelo(features_scaled)}")

optimal_k, inertias, silhouette_scores= cluster_cotovelo(features_scaled)


📈 Aplicando Método do Cotovelo...
✅ Número ótimo de clusters: (2, [14.896031994685753, inf], [0])


In [ ]:
from sklearn.cluster import KMeans

def clusters(features_scaled, optimal_k):

    kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(features_scaled)

    return kmeans, clusters

clusters, k_means = clusters(features_scaled, optimal_k)

In [123]:
k_means

array([1, 0, 0], dtype=int32)

In [124]:
def clusters_df(data, k_means, features, final_retorno):

    df_clusters = pd.DataFrame({
        'Ticker': data['Close'].columns,
        'Cluster': k_means
    })

    df_combinado = pd.concat([
        df_clusters.set_index('Ticker'),
        features,
        pd.Series(final - 1, name='Retorno_Total', index=data['Close'].columns)
    ], axis=1)

    return df_combinado

df_combinado = clusters_df(data,k_means,features,final)
df_combinado

,Cluster,Retorno_Medio,Volatilidade,Sharpe,Retorno_Acumulado,Skewness,Kurtosis,Max_Drawdown,Corr_Media,Retorno_Total
Ticker,,,,,,,,,,
BOVA11.SA,1,0.039602,0.260286,0.152149,0.026523,-0.967831,17.580253,0.0,0.416212,0.026523
IVVB11.SA,0,0.240578,0.222593,1.080801,1.899715,0.442009,7.259936,0.0,0.416212,1.899715
SMAL11.SA,0,-0.046465,0.318633,-0.145828,-0.383871,-1.191528,11.766888,0.0,0.416212,-0.383871


In [ ]:
def portfolio(returns,data, df_combinado, optimal_k):

    tickers_selecionados = []
    for cluster_id in range(optimal_k):
        cluster_data = df_combinado[df_combinado['Cluster'] == cluster_id]
        if len(cluster_data) > 0:
            # Selecionar o melhor de cada cluster
            best_ticker = cluster_data['Retorno_Total'].idxmax()
            tickers_selecionados.append(best_ticker)
            print(f"Cluster {cluster_id}: {best_ticker} (Retorno: {cluster_data.loc[best_ticker, 'Retorno_Total']*100:.2f}%)")

    if not tickers_selecionados:
        tickers_selecionados = list(data.columns[:3])
    
    portfolio_all = returns.mean(axis=1)
    retorno_acumulado_all = (1 + portfolio_all).cumprod()

    # Carteira selecionada
    if len(tickers_selecionados) > 0:
        dados_selecionados = data['Close'][tickers_selecionados]
        weights = [1/len(tickers_selecionados)] * len(tickers_selecionados)
        portfolio_selected = dados_selecionados.pct_change().dropna().dot(weights)
        retorno_acumulado_selected = (1 + portfolio_selected).cumprod()
    else:
        portfolio_selected = portfolio_all
        retorno_acumulado_selected = retorno_acumulado_all
        
    return tickers_selecionados, retorno_acumulado_all, retorno_acumulado_selected

tickers_selecionados, retorno_acumulado_all, retorno_acumulado_selected = portfolio(returns, data,df_combinado, optimal_k)



💼 SELEÇÃO DE PORTFÓLIO OTIMIZADO
Cluster 0: IVVB11.SA (Retorno: 189.97%)
Cluster 1: BOVA11.SA (Retorno: 2.65%)


In [136]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
def gerar_graficos(features, clusters, retorno,
                retorno_acumulado_all, retorno_acumulado_selected, 
                df_combinado, silhouette_scores, K, inertias,
                optimal_k, tickers_selecionados
                ):
    """
    Gera 9 gráficos diferentes e os salva na pasta 'static'.
    """
    os.makedirs('static', exist_ok=True)  # Garante que a pasta 'static' existe

    # 1. Método do Cotovelo
    plt.figure(figsize=(8, 6))
    if len(K) > 1 and len(inertias) > 1:
        plt.plot(K, inertias, 'bo-', linewidth=2)
        plt.axvline(x=optimal_k, color='red', linestyle='--', label=f'K ótimo = {optimal_k}')
        plt.xlabel('Número de Clusters')
        plt.ylabel('Inércia')
        plt.title('Método do Cotovelo')
        plt.legend()
        plt.grid(True, alpha=0.3)
    else:
        plt.text(0.5, 0.5, f'K = {optimal_k}', ha='center', va='center', fontsize=14)
        plt.title('Clusters')
    plt.tight_layout()
    plt.savefig('static/grafico_cotovelo.png')
    plt.close()

    # 2. PCA Visualization
    plt.figure(figsize=(8, 6))
    if features.shape[0] > 1 and features.shape[1] > 1:
        try:
            pca = PCA(n_components=2)
            features_pca = pca.fit_transform(features)
            scatter = plt.scatter(features_pca[:, 0], features_pca[:, 1], 
                                   c=clusters, cmap='viridis', s=100, alpha=0.7)
            plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
            plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
            plt.title('Visualização PCA dos Clusters')
            plt.colorbar(scatter)
        except:
            plt.text(0.5, 0.5, 'PCA não disponível', ha='center', va='center')
            plt.title('PCA')
    else:
        plt.text(0.5, 0.5, 'Dados insuficientes', ha='center', va='center')
        plt.title('PCA')
    plt.tight_layout()
    plt.savefig('static/grafico_pca.png')
    plt.close()

    # 3. Comparação de Carteiras
    plt.figure(figsize=(8, 6))
    if not retorno_acumulado_all.empty:
        retorno_acumulado_all.plot(label='Todos os Tickers', linewidth=2)
        retorno_acumulado_selected.plot(label='Seleção por Clusters', linewidth=2, color='red')
        plt.title('Comparação de Desempenho')
        plt.ylabel('Retorno Acumulado')
        plt.legend()
        plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('static/grafico_comparacao_carteiras.png')
    plt.close()

    # 4. Distribuição de Retornos
    plt.figure(figsize=(8, 6))
    if not retorno.empty:
        retorno.boxplot(rot=45)
        plt.title('Distribuição de Retornos')
        plt.ylabel('Retorno Diário')
        plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.savefig('static/grafico_distribuicao_retornos.png')
    plt.close()

    # 5. Matriz de Correlação
    plt.figure(figsize=(8, 6))
    if len(tickers_selecionados) > 1 and not retorno.empty:
        corr_selected = retorno[tickers_selecionados].corr()
        sns.heatmap(corr_selected, annot=True, fmt='.2f', cmap='coolwarm', 
                    vmin=-1, vmax=1, cbar_kws={'shrink': 0.8})
        plt.title('Correlação - Carteira Selecionada')
    else:
        plt.text(0.5, 0.5, 'N/A', ha='center', va='center')
        plt.title('Correlação')
    plt.tight_layout()
    plt.savefig('static/grafico_correlacao.png')
    plt.close()

    # 6. Risk-Return Map
    plt.figure(figsize=(8, 6))
    if not df_combinado.empty:
        for cluster_id in range(optimal_k):
            cluster_data = df_combinado[df_combinado['Cluster'] == cluster_id]
            if len(cluster_data) > 0:
                plt.scatter(cluster_data['Volatilidade']*100, 
                            cluster_data['Retorno_Medio']*100,
                            label=f'Cluster {cluster_id}', s=100, alpha=0.7)
        plt.xlabel('Volatilidade Anual (%)')
        plt.ylabel('Retorno Anual (%)')
        plt.title('Mapa Risco-Retorno')
        plt.legend()
        plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('static/grafico_risco_retorno.png')
    plt.close()
    '''
    # 7. Silhouette Scores
    plt.figure(figsize=(8, 6))
    if len(silhouette_scores) > 1:
        plt.plot(K[:len(silhouette_scores)], silhouette_scores, 'go-', linewidth=2)
        plt.axvline(x=optimal_k, color='red', linestyle='--')
        plt.xlabel('Número de Clusters')
        plt.ylabel('Silhouette Score')
        plt.title('Silhouette Score')
        plt.grid(True, alpha=0.3)
    else:
        plt.text(0.5, 0.5, 'N/A', ha='center', va='center')
        plt.title('Silhouette Score')
    plt.tight_layout()
    plt.savefig('static/grafico_silhouette.png')
    plt.close()
    '''
    # 8. Performance Metrics
    plt.figure(figsize=(8, 6))
    try:
        metrics = pd.DataFrame({
            'Todos': [
                (retorno_acumulado_all.iloc[-1] - 1) * 100,
                retorno.std() * np.sqrt(252) * 100,
                (retorno.mean() / retorno.std()) * np.sqrt(252) if retorno.std() > 0 else 0
            ],
            'Selecionados': [
                (retorno_acumulado_selected.iloc[-1] - 1) * 100,
                retorno.std() * np.sqrt(252) * 100,
                (retorno.mean() / retorno.std()) * np.sqrt(252) if retorno.std() > 0 else 0
            ]
        }, index=['Retorno Total (%)', 'Volatilidade (%)', 'Sharpe Ratio'])
        
        metrics.plot(kind='bar', alpha=0.8)
        plt.title('Métricas de Performance')
        plt.ylabel('Valor')
        plt.legend(title='Carteira')
        plt.grid(True, alpha=0.3, axis='y')
        plt.xticks(rotation=45, ha='right')
    except:
        plt.text(0.5, 0.5, 'Métricas não disponíveis', ha='center', va='center')
        plt.title('Métricas')
    plt.tight_layout()
    plt.savefig('static/grafico_metricas.png')
    plt.close()

    # 9. Composição da Carteira
    plt.figure(figsize=(8, 6))
    if len(tickers_selecionados) > 0:
        sizes = [100/len(tickers_selecionados)] * len(tickers_selecionados)
        colors = plt.cm.Set3(range(len(tickers_selecionados)))
        plt.pie(sizes, labels=tickers_selecionados, colors=colors, autopct='%1.1f%%')
        plt.title('Composição da Carteira Otimizada')
    else:
        plt.text(0.5, 0.5, 'Sem seleção', ha='center', va='center')
        plt.title('Composição')
    plt.tight_layout()
    plt.savefig('static/grafico_composicao_carteira.png')
    plt.close()

In [141]:
features

,Retorno_Medio,Volatilidade,Sharpe,Retorno_Acumulado,Skewness,Kurtosis,Max_Drawdown,Corr_Media
Ticker,,,,,,,,
BOVA11.SA,0.039602,0.260286,0.152149,0.026523,-0.967831,17.580253,0.0,0.416212
IVVB11.SA,0.240578,0.222593,1.080801,1.899715,0.442009,7.259936,0.0,0.416212
SMAL11.SA,-0.046465,0.318633,-0.145828,-0.383871,-1.191528,11.766888,0.0,0.416212


In [150]:
gerar_graficos(features, clusters,returns,
                retorno_acumulado_all, retorno_acumulado_selected, 
                df_combinado, silhouette_scores, list(range(1, optimal_k + 1)), inertias,
                optimal_k, tickers_selecionados
                )

In [ ]:
def resultados(returns):
    print("\n" + "=" * 70)
    print("🎯 RELATÓRIO FINAL")
    print("=" * 70)

    print(f"\n📊 Resumo da Análise:")
    print(f"   • Ações analisadas: {len(data.columns)}")
    print(f"   • Clusters identificados: {optimal_k}")
    print(f"   • Ações selecionadas: {len(tickers_selecionados)}")

    if not returns.empty:
        ret_all = (retorno_acumulado_all.iloc[-1] - 1) * 100
        ret_sel = (retorno_acumulado_selected.iloc[-1] - 1) * 100
        
        print(f"\n📈 Performance:")
        print(f"   • Carteira Completa: {ret_all:.2f}%")
        print(f"   • Carteira Otimizada: {ret_sel:.2f}%")
        
        if ret_sel > ret_all:
            print(f"   ✅ A seleção por clusters melhorou o retorno em {ret_sel - ret_all:.2f}%")
        else:
            print(f"   ⚠️ A carteira completa teve melhor desempenho")

    print(f"\n💼 Carteira Otimizada:")
    for i, ticker in enumerate(tickers_selecionados, 1):
        peso = 100 / len(tickers_selecionados)
        print(f"   {i}. {ticker}: {peso:.1f}%")

    print("\n✅ Análise concluída com sucesso!")
    print("=" * 70)
resultados(returns,)

In [ ]:
def otimizar_portfolio(ativos, metodo, prazo_teste):

SyntaxError: expected ':' (2623691959.py, line 1)

In [ ]:
def calcular_metricas(returns, periodo_meses):
    """
    Calcula retorno esperado e volatilidade ajustados para o período de X meses.
    """
    dias_por_mes = 21  # Aproximadamente 21 dias úteis por mês
    periodo_dias = periodo_meses * dias_por_mes

    retorno_esperado = (1 + returns.mean())**periodo_dias - 1
    volatilidade = returns.std() * np.sqrt(periodo_dias)

    return retorno_esperado, volatilidade

def portfolio_metrics(weights):
    """
    Calcula retorno, risco e Sharpe do portfólio.
    """
    weights = np.array(weights)
    retorno = np.sum(weights * retorno_esperado)
    risco = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = retorno / risco
    return retorno, risco, sharpe


def otimizar_portfolio(returns, metodo, periodo_meses):
    """
    Otimiza a carteira com base no método escolhido (mínimo risco ou máximo Sharpe).
    """
    retorno_esperado, cov_matrix = calcular_metricas(returns, periodo_meses)
    n_ativos = len(retorno_esperado)



    if metodo == 'min_risco':
        # Otimizar para mínima volatilidade
        def objetivo(weights):
            _, risco, _ = portfolio_metrics(weights)
            return risco
    elif metodo == 'max_sharpe':
        # Otimizar para máximo Sharpe
        def objetivo(weights):
            _, _, sharpe = portfolio_metrics(weights)
            return -sharpe

    # Restrições e limites
    constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]  # Soma dos pesos = 1
    bounds = [(0, 1) for _ in range(n_ativos)]  # Sem vendas a descoberto

    # Otimização
    resultado = minimize(objetivo, x0=np.ones(n_ativos) / n_ativos, bounds=bounds, constraints=constraints)
    pesos_otimizados = resultado.x

    return pesos_otimizados, portfolio_metrics(pesos_otimizados)

In [ ]:
import gurobipy as gp
from gurobipy import GRB
from math import sqrt
import pandas as pd
import numpy as np


def min_risk_portfolio(tickers, start_date='2022-01-01', end_date='2024-01-01'):

    stocks = tickers
    data = yf.download(stocks, period='2y')


    closes = np.transpose(np.array(data.Close)) # matrix of daily closing prices
    absdiff = np.diff(closes)                   # change in closing price each day
    reldiff = np.divide(absdiff, closes[:,:-1]) # relative change in daily closing price
    delta = np.mean(reldiff, axis=1)            # mean price change
    sigma = np.cov(reldiff)                     # covariance (standard deviations)
    std = np.std(reldiff, axis=1)               # standard deviation



    # Create an empty model
    m = gp.Model('portfolio')

    # Add matrix variable for the stocks
    x = m.addMVar(len(stocks))

    # Objective is to minimize risk (squared).  This is modeled using the
    # covariance matrix, which measures the historical correlation between stocks
    portfolio_risk = x @ sigma @ x
    m.setObjective(portfolio_risk, GRB.MINIMIZE)

    # Fix budget with a constraint
    m.addConstr(x.sum() == 1, 'budget')

    # Verify model formulation
    m.write('portfolio_selection_optimization.lp')

    # Optimize model to find the minimum risk portfolio
    m.optimize()


    minrisk_volatility = sqrt(m.ObjVal)
    minrisk_return = delta @ x.X
    pd.DataFrame(data=np.append(x.X, [minrisk_volatility, minrisk_return]),
                index=stocks + ['Volatility', 'Expected Return'],
                columns=['Minimum Risk Portfolio'])

min_risk_portfolio([
        "VALE3.SA",   # Mineração / Commodities
        "ITUB4.SA",   # Bancos / Financeiro
        "WEGE3.SA",   # Indústria (WEG)
        "RADL3.SA",   # Petróleo & Gás (PetroRio)
        "QUAL3.SA",   # Qualicorp (Saúde)
        ])

C:\Users\gabri\AppData\Local\Temp\ipykernel_2880\4246164385.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, period='2y')
[*********************100%***********************]  5 of 5 completed

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads



Optimize a model with 1 rows, 5 columns and 5 nonzeros
Model fingerprint: 0x0ad8762f
Model has 15 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e-04, 4e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 1 rows, 5 columns, 5 nonzeros
Presolved model has 15 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 4
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   6.18218902e+03 -6.18218902e+03  5.00e+03 2.84e-06  1.00e+06     0s
   1   6.31338209e-03 -7.80613958e+00  4.89e+00 2.78e-09  1.08e+03     0s
   2   1.87815976e-04 -6.42988103e+00  4.89e-06 2.77e-15  8.23e+01     0s
   3   1.87780006e-04 -6.44025755e-03  1.

In [ ]:
stocks = [
    "VALE3.SA",   # Mineração / Commodities
    "ITUB4.SA",   # Bancos / Financeiro
    "WEGE3.SA",   # Indústria (WEG)
    "RADL3.SA",   # Petróleo & Gás (PetroRio)
    "QUAL3.SA",   # Qualicorp (Saúde)
    ]


data

C:\Users\gabri\AppData\Local\Temp\ipykernel_2880\939634966.py:9: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, period='2y')
[*********************100%***********************]  5 of 5 completed


Price           Close                                                  High  \
Ticker       ITUB4.SA  QUAL3.SA   RADL3.SA   VALE3.SA   WEGE3.SA   ITUB4.SA   
Date                                                                          
2023-10-03  20.812450  2.861332  26.310642  54.966007  33.867821  21.070649   
2023-10-04  21.289734  2.921151  26.873333  54.379852  33.858189  21.383625   
2023-10-05  21.626171  2.921151  26.837505  54.404621  33.684856  21.774832   
2023-10-06  21.829601  2.881272  27.148548  55.197163  33.511517  22.134746   
2023-10-09  21.641823  2.901211  26.963867  54.800888  33.559669  21.712240   
...               ...       ...        ...        ...        ...        ...   
2025-09-29  38.872433  2.510000  18.100000  57.279999  36.369999  39.342222   
2025-09-30  39.052349  2.530000  18.420000  57.580002  36.590000  39.482155   
2025-10-01  38.349998  2.480000  18.240000  58.310001  35.790001  39.410000   
2025-10-02  37.930000  2.400000  18.020000  58.700001  36.130001  38.700001   
2025-10-03  38.209999  2.330000  18.830000  58.590000  36.400002  38.209999   

Price                                                  ...       Open  \
Ticker      QUAL3.SA   RADL3.SA   VALE3.SA   WEGE3.SA  ...   ITUB4.SA   
Date                                                   ...              
2023-10-03  2.980970  26.698704  55.279719  34.792279  ...  20.992408   
2023-10-04  2.931120  26.999452  55.131112  34.233748  ...  20.890698   
2023-10-05  2.990939  27.012469  54.858677  34.195232  ...  21.250608   
2023-10-06  2.891241  27.352672  55.684242  33.848560  ...  21.430566   
2023-10-09  2.921151  27.177711  54.800888  33.733005  ...  21.633997   
...              ...        ...        ...        ...  ...        ...   
2025-09-29  2.550000  18.100000  57.750000  36.970001  ...  39.062349   
2025-09-30  2.550000  18.799999  57.779999  36.790001  ...  39.382202   
2025-10-01  2.530000  18.600000  58.669998  36.360001  ...  39.259998   
2025-10-02  2.510000  18.200001  58.860001  36.349998  ...  38.419998   
2025-10-03  2.420000  18.840000  59.119999  37.150002  ...  37.910000   

Price                                                    Volume           \
Ticker      QUAL3.SA   RADL3.SA   VALE3.SA   WEGE3.SA  ITUB4.SA QUAL3.SA   
Date                                                                       
2023-10-03  2.951060  26.494971  54.553226  34.686351  23494130  4645900   
2023-10-04  2.871302  26.407656  54.817400  33.993004  26187480  3501900   
2023-10-05  2.921151  26.837505  54.379855  33.993008  31498940  3517800   
2023-10-06  2.891241  26.818061  54.396365  33.511517  48554660  4629600   
2023-10-09  2.841392  27.080511  54.264277  33.395966  16062750  3578400   
...              ...        ...        ...        ...       ...      ...   
2025-09-29  2.510000  18.000000  57.500000  36.970001  18567400  2798200   
2025-09-30  2.490000  18.270000  57.669998  36.599998  32294600  1541300   
2025-10-01  2.530000  18.549999  57.759998  36.290001  24775900  2226500   
2025-10-02  2.460000  18.190001  58.360001  35.869999  16596700  2141600   
2025-10-03  2.420000  17.900000  58.700001  36.200001  17249500  4584700   

Price                                     
Ticker      RADL3.SA  VALE3.SA  WEGE3.SA  
Date                                      
2023-10-03   4918300  15268800   7549900  
2023-10-04   3847500  14377000   5224100  
2023-10-05   4269400  11556300   4103700  
2023-10-06   5474300  22711400   6279600  
2023-10-09   4043800  14392200   3984400  
...              ...       ...       ...  
2025-09-29  15732800  15580200   4873200  
2025-09-30  11312900  18280700   7668000  
2025-10-01   6612900  18278300  11275700  
2025-10-02   5508400  14824500   8501800  
2025-10-03   9499000  19240200   7971200  

[501 rows x 25 columns]

In [151]:
# Create an expression representing the expected return for the portfolio

def fronteira_eficiente(tickers):
    stocks = tickers

    data = yf.download(stocks, period='2y')

    closes = np.transpose(np.array(data.Close)) # matrix of daily closing prices
    absdiff = np.diff(closes)                   # change in closing price each day
    reldiff = np.divide(absdiff, closes[:,:-1]) # relative change in daily closing price
    delta = np.mean(reldiff, axis=1)            # mean price change
    sigma = np.cov(reldiff)                     # covariance (standard deviations)
    std = np.std(reldiff, axis=1)               # standard deviation

    portfolio_return = delta @ x
    target = m.addConstr(portfolio_return == minrisk_return, 'target')

    # Solve for efficient frontier by varying target return
    frontier = np.empty((2,0))
    for r in np.linspace(delta.min(), delta.max(), 25):
        target.rhs = r
        m.optimize()
        frontier = np.append(frontier, [[sqrt(m.ObjVal)],[r]], axis=1)

    #plt.figure(figsize=(10,10))

    fig, ax = plt.subplots(figsize=(10,8))

    # Plot volatility versus expected return for individual stocks
    ax.scatter(x=std, y=delta,
            color='Blue', label='Individual Stocks')
    for i, stock in enumerate(stocks):
        ax.annotate(stock, (std[i], delta[i]))

    # Plot volatility versus expected return for minimum risk portfolio
    ax.scatter(x=minrisk_volatility, y=minrisk_return, color='DarkGreen')
    ax.annotate('Minimum\nRisk\nPortfolio', (minrisk_volatility, minrisk_return),
                horizontalalignment='right')

    # Plot efficient frontier
    ax.plot(frontier[0], frontier[1], label='Efficient Frontier', color='DarkGreen')

    # Format and display the final plot
    ax.axis([frontier[0].min()*0.7, frontier[0].max()*1.3, delta.min()*1.2, delta.max()*1.2])
    ax.set_xlabel('Volatility (standard deviation)')
    ax.set_ylabel('Expected Return')
    ax.legend()
    ax.grid()
    plt.show()

fronteira_eficiente([
    "VALE3.SA",   # Mineração / Commodities
    "ITUB4.SA",   # Bancos / Financeiro
    "WEGE3.SA",   # Indústria (WEG)
    "RADL3.SA",   # Petróleo & Gás (PetroRio)
    "QUAL3.SA",   # Qualicorp (Saúde)
    ])

C:\Users\gabri\AppData\Local\Temp\ipykernel_2880\3559034462.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, period='2y')
[*********************100%***********************]  5 of 5 completed

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 5 columns and 20 nonzeros
Model fingerprint: 0xc406c55f


Model has 15 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e-04, 4e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-04, 1e+00]
Presolve time: 0.01s

Barrier solved model in 0 iterations and 0.01 seconds (0.00 work units)
Model is infeasible or unbounded


AttributeError: Unable to retrieve attribute 'ObjVal'

In [ ]:
# =====================================
# CÁLCULO DE RETORNOS E MÉTRICAS
# =====================================
print("\n📈 Calculando retornos e métricas...")

# Calcular retornos
retorno = data.pct_change().dropna()

if retorno.empty or len(retorno) < 10:
    print("⚠️ Poucos dados de retorno. Ajustando análise...")
    # Garantir mínimo de dados
    if retorno.empty:
        retorno = pd.DataFrame(np.random.randn(100, len(data.columns)) * 0.01, 
                               columns=data.columns)

# Calcular métricas básicas
retorno_acumulado = (1 + retorno).cumprod()
final_retorno_acumulado = retorno_acumulado.iloc[-1] if not retorno_acumulado.empty else pd.Series(1, index=data.columns)
